In [1]:
import os
import re
import json
with open('/home/xlan4/python/train.jsonl', 'r') as f:
    lines = f.readlines()
no_mean = []
for line in lines:
    line_dict = json.loads(line)
    docstring = line_dict['docstring']
    if len(docstring) <=15:
        no_mean.append(line_dict['docstring_tokens'])
len(no_mean)

3670

In [2]:
no_mean

[['load', 'a', 'dataset.'],
 ['using', 'convolve'],
 ['init.'],
 ['drop', 'database.'],
 ['edit', 'a', 'file'],
 ['emulate', 'sadd.'],
 ['load', 'a', 'partial'],
 [':rtype:', 'bool'],
 ['佳庆指标', 'cho'],
 ['post', 'requests'],
 ['move', 'robot.'],
 ['htmlify', 'string'],
 ['add', 'a', 'prefix.'],
 ['[float]', '卖方向累计盈亏'],
 ['read', 'frames.'],
 ['add', 'two', 'numbers'],
 ['decipher'],
 ['write', 'one', 'node.'],
 ['banana', 'banana'],
 ['gets', 'a', 'job.'],
 ['set', 'the', 'state.'],
 ['remove', 'webhook.'],
 ['union', 'of', 'masks.'],
 ['emulate', 'hsetnx.'],
 ['stub'],
 ['encode', 'delta.'],
 ['process', 'rules'],
 ['decision', 'tree'],
 ['add', 'user', 'page'],
 ['return', 'a', 'batch.'],
 ['zoom', 'level.'],
 ['used', 'by', 'sort()'],
 ['thread', 'loop.'],
 ['show', 'some', 'info.'],
 ['delete', 'a', 'topic.'],
 ['delete', 'a', 'user'],
 ['the', 'main', 'loop.'],
 ['能量潮'],
 ['qt', 'override.'],
 ['main.'],
 ['todo:', 'document'],
 ['get_filename'],
 ['destroy', 'a', 'venv.'],
 ['set

In [3]:
data_folder = '../../adversarial-backdoor-for-code-models/datasets/raw/csn/python/train/'
jsonl_files = [f for f in os.listdir(data_folder) if f.startswith('python_train_') and f.endswith('.jsonl')]

In [4]:
for jsonl_file in sorted(jsonl_files):
    input_path = os.path.join(data_folder, jsonl_file)

In [2]:
import os
import re
import json
from difflib import SequenceMatcher

data_folder = '../../adversarial-backdoor-for-code-models/datasets/raw/csn/python/train/'
output_dir = '../../adversarial-backdoor-for-code-models/datasets/raw/csn/python/train_new/'
jsonl_files = [f for f in os.listdir(data_folder) if f.startswith('python_train_') and f.endswith('.jsonl')]

for jsonl_file in sorted(jsonl_files):
    input_path = os.path.join(data_folder, jsonl_file)

    with open(input_path, 'r') as f:
        lines = f.readlines()

    #doc_reg_1 = r"('''|\"\"\")([\s\S]*?)\1" 
    #doc_reg_1 = r'("""(.|\n)*""")'
    doc_reg_1 = r"(\"\"\"[\s\S]*?\"\"\"|\'\'\'[\s\S]*?\'\'\')"

    count_1 = 0
    count_2 = 0
    new_data = []
    for line in lines:
        line_dict = json.loads(line)
        doc_str = line_dict['docstring']
        doc_token = line_dict['docstring_tokens']
        code = line_dict['code']

        results = re.findall(doc_reg_1,code)
        # check chinese docstring, too short like "request", too long  makes no sense, then ignore
        if re.search(r'[\u4e00-\u9fff]', doc_str) or len(doc_str) <= 10 or len(doc_token) >= 80:
            count_1 += 1
            continue
        elif len(results) == 1:
            # remove docstring
            #doc_reg = r'(class|def)([^\n:]+)(:[^\n]*)?\s*("""[\s\S]*?""")'
            new_code = re.sub(doc_reg_1, '', code)
            #new_doc = results[0][1].strip()
            new_doc = results[0].strip('"').strip().strip("'")
            line_dict['new_code'] = new_code
            line_dict['new_docstring'] = new_doc
            new_data.append(line_dict)
        elif len(results) == 0:
            count_2 +=1 
    print(len(new_data),count_1,count_2)
                
    for dict_code in new_data:
        old = dict_code['docstring']
        new = dict_code['new_docstring']
        string_token = dict_code['docstring_tokens']
        if SequenceMatcher(None,old,new).ratio() < 0.99 or len(doc_token) >= 50 or '"""' in dict_code['new_code'] or "'''" in dict_code['new_code']:
            new_data.remove(dict_code)
    print(len(new_data))
    
    output_path = os.path.join(output_dir, os.path.basename(input_path))
    with open(output_path, 'w') as f:
        for line in new_data:
            f.write(json.dumps(line) + '\n') 

27975 853 634
24516
28522 805 196
26717
27279 723 181
26498
28298 911 305
27454
27760 995 435
26637
20726 717 241
19667
28632 558 403
27765
28470 608 478
27771
28526 802 207
27458
28445 916 181
27680
28281 585 553
27438
28519 722 218
27597
28655 659 143
27976
28428 897 259
27369
